In [0]:
%sql
SET datasets.path=dbfs:/mnt/demo-datasets/bookstore;

key,value
datasets.path,dbfs:/mnt/demo-datasets/bookstore


In [0]:
%sql 
-- Bronze Layer Tables
CREATE OR REFRESH STREAMING LIVE TABLE books_bronze
COMMENT "The raw data, ingested from cdc feed"
AS 
  SELECT
    * 
  FROM 
    cloud_files("${datasets.path}/books-cdc", "json");

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


In [0]:
%sql
-- Silver Table
CREATE OR REFRESH STREAMING LIVE TABLE books_silver;

APPLY CHANGES INTO LIVE.books_silver
  FROM STREAM(LIVE.books_bronze)
  KEYS(book_id)
  APPLY AS DELETE WHEN row_status = "DELETE"
  SEQUENCE BY row_time
  COLUMNS * EXCEPT (row_status, row_time)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1415010210644092> in <cell line: 1>()
      6     display(df)
      7     return df
----> 8   _sqldf = ____databricks_percent_sql()
      9 finally:
     10   del ____databricks_percent_sql

<command-1415010210644092> in ____databricks_percent_sql()
      3     import base64
      4     spark.sql(base64.standard_b64decode("LS0gU2lsdmVyIFRhYmxlCkNSRUFURSBPUiBSRUZSRVNIIFNUUkVBTUlORyBMSVZFIFRBQkxFIGJvb2tzX3NpbHZlcg==").decode())
----> 5     df = spark.sql(base64.standard_b64decode("QVBQTFkgQ0hBTkdFUyBJTlRPIExJVkUuYm9va3Nfc2lsdmVyCiAgRlJPTSBTVFJFQU0oTElWRS5ib29rc19icm9uemUpCiAgS0VZUyhib29rX2lkKQogIEFQUExZIEFTIERFTEVURSBXSEVOIHJvd19zdGF0dXMgPSAiREVMRVRFIgogIFNFUVVFTkNFIEJZIHJvd190aW1lCiAgQ09MVU1OUyAqIEVYQ0VQVCAocm93X3N0YXR1cywgcm93X3RpbWUp").decode())
      6     display(df)
      7     return df

/databricks/spark/python/pyspark/in

In [0]:
%sql
-- Gold Table
CREATE LIVE TABLE author_counts_state
  COMMENT "author of books per author"
AS 
  SELECT 
    author, 
    count(*) no_books,
    current_timestamp() updated_time
  FROM 
    LIVE.books_silver
  GROUP BY 
    author 

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


In [0]:
%sql
-- DLT Views
CREATE LIVE VIEW books_sales
AS 
  SELECT 
    b.title,
    o.quantity
  FROM (
    SELECT 
      *,
      EXPLODE(books) AS book
    FROM 
      LIVE.orders_cleaned
  ) as o INNER JOIN LIVE.books_silver b
  ON o.book.book_id = b.book_id

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate tables reading from this view."
